# **Memorial de Cálculo - Trabalho 1**

**Disciplina:** PPGEE0017 - Projeto de Circuitos Integrados

**Discente:** André Paiva Conrado Rodrigues (N=2)

## 1 - Escopo do trabalho


### 1.1 - Características do circuito a ser dimensionado

Dimensionamento de Fontes de Corrente (simples e cascode) com os seguintes parâmetros:

| Parâmetro | Min | Typ  | Máx   | Unidade |
|-----------|-----|------|-------|---------|
| $I_{out}$ | --  | 20   | --    | $\mu \mathrm{A}$ |
| $V_{out}$ | --  | --   | 0.3   | V       |
| $V_{DD}$  | --  | 0.6  | --    | V       |
| $V_{SS}$  | --  | -0.6 | --    | V       |
| $W$       | 160 | --   | 32000 | nm      |
| $L$       | 120 | --   | 24000 | nm      |

### 1.2 - Parâmetros da tecnologia

| Parâmetro  | Canal N | Canal P | Unidade |
|------------|---------|---------|---------|
| $I_{sq}$   | 353.38  | 49.514  | nA      |
| $n$        | 1,3790  | 1,2952  | -       |
| $\phi_{t}$ | 26      | 26      | mV      |
| $V_{T0}$   | 0,3618  | -0,3554 | V       |